In [1]:
# # 3. Analítica (Capa Oro)
# Agregaciones de negocio.

# %%
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as _sum, desc
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder
    .appName("Lab_SECOP_Gold")
    .master("spark://spark-master:7077")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e27ebc2b-2dc9-4d66-b2bb-076c6fbbe9f8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 185ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

In [2]:
# Leer Plata 
df_silver = spark.read.format("delta").load("file:/app/data/lakehouse/silver/secop")

# %%
# Agregación (Top 10 departamentos por valor contratado)
df_gold = (
    df_silver
    .groupBy("departamento")
    .agg(_sum("precio_base").alias("total_contratado"))
    .orderBy(desc("total_contratado"))
    .limit(10)
)


26/02/02 02:18:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/02 02:18:31 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/02 02:18:46 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
                                                                                

In [3]:
# Persistir Oro
df_gold.write.format("delta").mode("overwrite").save("file:/app/data/lakehouse/gold/top_deptos")

# %%
# Visualizar 
print("Top 10 Departamentos por contratación:")
df_gold.show(truncate=False)

26/02/02 02:19:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

Top 10 Departamentos por contratación:
+------------+----------------+
|departamento|total_contratado|
+------------+----------------+
|VALLE       |394161052301    |
|BOLIVAR     |386120570040    |
|ATLANTICO   |384507061234    |
|CUNDINAMARCA|378536228613    |
|ANTIOQUIA   |351958791420    |
|SANTANDER   |343270735552    |
|BOGOTA      |326788630428    |
+------------+----------------+

